# 1. Set Up

In [5]:
pip install python-binance

Note: you may need to restart the kernel to use updated packages.


# 2. Authenticate

In [2]:
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd
import numpy as np
import math

In [3]:
%store -r byron_api_key
%store -r byron_api_secret

In [4]:
client = Client(byron_api_key, byron_api_secret)

# 3. Get Historical Prices

In [5]:
import requests
import json
import datetime
import pytz

### Set Up Endpoint

In [6]:
# Define the symbols you want to retrieve data for
#symbols = ["BTCUSDT", "ETHUSDT", "LINKUSDT"]
symbols = ['BTCUSDT', 'ETHUSDT','BNBUSDT','XRPUSDT','BUSDUSDT','ADAUSDT','DOGEUSDT','MATICUSDT','SOLUSDT','DOTUSDT','SHIBUSDT','LTCUSDT','TRXUSDT','AVAXUSDT','UNIUSDT','ATOMUSDT','LINKUSDT','ETCUSDT','XMRUSDT','BCHUSDT','HBARUSDT','XLMUSDT','APTUSDT','FILUSDT','LDOUSDT','NEARUSDT','APEUSDT','ALGOUSDT','VETUSDT','QNTUSDT','ICPUSDT','GRTUSDT','MANAUSDT','FTMUSDT','EOSUSDT','AAVEUSDT','SANDUSDT','EGLDUSDT','FLOWUSDT','THETAUSDT','AXSUSDT','XTZUSDT','LUNCUSDT','CHZUSDT','MINAUSDT','IMXUSDT','FXSUSDT','ZECUSDT','CAKEUSDT','MKRUSDT']
# Define the API endpoint
endpoint = 'https://api.binance.com/api/v3/klines'
# Define the interval (1 minute, 3 minute, etc.)
interval = '1d'
# Define the number of data points you want to retrieve
#limit = 150

start_date = datetime.datetime(2022, 3, 2) #Starting the "2022-01-01"
start_timestamp = int(start_date.timestamp() * 1000)

end_date = datetime.datetime.now().date() - datetime.timedelta(days=0) #Ending yesterday
end_timestamp = int(datetime.datetime.combine(end_date, datetime.time()).timestamp() * 1000)


### Request Data

In [7]:
# Initialize an empty list to store the data
data = []

# Loop through each symbol and retrieve the data
for symbol in symbols:
  #url = f'{endpoint}?symbol={symbol}&interval={interval}&limit={limit}'
  url = f'{endpoint}?symbol={symbol}&interval={interval}&startTime={start_timestamp}&endTime={end_timestamp}' 
  response = requests.get(url)
  # Convert the response data to a Pandas dataframe
  df = pd.DataFrame(response.json(), columns=[ 'Open_Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'])
  # Add the symbol column to the dataframe
  df['Symbol'] = symbol
  # Append the dataframe to the data list
  data.append(df)
df = pd.concat(data, axis=0, ignore_index=True)


### Select the Price Column and Rename

In [15]:
f = []
for symbol in symbols:
    n_df  = df[df["Symbol"] == symbol]
    #n_df.rename(columns= {"Close": "Price_"+symbol}, inplace=True)
    n_df.rename(columns= {"Close": symbol}, inplace=True)
    #n_df = n_df[["Open_Time","Price_"+symbol]] #Rename the column
    n_df = n_df[["Open_Time",symbol]]
    n_df['Open_Time'] = pd.to_datetime(n_df['Open_Time']/1000, unit='s') #Convert Open_Time to date
    #n_df["Price_"+symbol] = n_df["Price_"+symbol].apply(pd.to_numeric) #Convert to numberic
    n_df[symbol] = n_df[symbol].apply(pd.to_numeric) #Convert to numberic
    f.append(n_df)

/var/folders/cg/vf981v4x5tz8b5szmpt19_xm0000gp/T/ipykernel_99876/2499673076.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  n_df.rename(columns= {"Close": symbol}, inplace=True)
/var/folders/cg/vf981v4x5tz8b5szmpt19_xm0000gp/T/ipykernel_99876/2499673076.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  n_df.rename(columns= {"Close": symbol}, inplace=True)
/var/folders/cg/vf981v4x5tz8b5szmpt19_xm0000gp/T/ipykernel_99876/2499673076.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

### DataFrame with Historical Prices

In [16]:
#n_df["Open_Time"]  = pd.to_datetime(n_df["Open_Time"]/1000, unit='s')
n_df = [n_df.set_index('Open_Time') for n_df in f]
n_df = pd.concat(n_df, axis=1)
hist_prices = n_df.copy()
hist_prices.reset_index(inplace=True) #Resetting the Date from Index to column again
hist_prices.head()

,Open_Time,BTCUSDT,ETHUSDT,BNBUSDT,XRPUSDT,BUSDUSDT,ADAUSDT,DOGEUSDT,MATICUSDT,SOLUSDT,...,AXSUSDT,XTZUSDT,LUNCUSDT,CHZUSDT,MINAUSDT,IMXUSDT,FXSUSDT,ZECUSDT,CAKEUSDT,MKRUSDT
0,2022-03-02,43892.98,2947.30,409.3,0.7687,0.9996,0.939,0.1330,1.637,100.68,...,55.53,3.492,NaN,0.1970,2.139,1.465,23.448,125.8,6.83,1995.0
1,2022-03-03,42454.00,2833.99,402.6,0.7515,1.0001,0.903,0.1297,1.586,95.62,...,53.31,3.393,NaN,0.1971,2.036,1.459,24.420,129.1,6.58,2039.0
2,2022-03-04,39148.66,2622.10,374.2,0.7127,0.9998,0.844,0.1226,1.478,88.32,...,49.79,3.082,NaN,0.1953,2.008,1.323,22.303,117.0,5.99,1796.0
3,2022-03-05,39397.96,2665.15,384.7,0.7540,0.9999,0.864,0.1250,1.508,89.63,...,51.89,3.146,NaN,0.2328,2.021,1.282,22.280,116.4,6.20,1839.0
4,2022-03-06,38420.81,2551.29,375.0,0.7251,0.9998,0.823,0.1207,1.435,84.57,...,48.89,2.973,NaN,0.2075,1.912,1.148,20.587,107.4,6.08,1719.0


In [17]:
hist_prices.tail(50)

,Open_Time,BTCUSDT,ETHUSDT,BNBUSDT,XRPUSDT,BUSDUSDT,ADAUSDT,DOGEUSDT,MATICUSDT,SOLUSDT,...,AXSUSDT,XTZUSDT,LUNCUSDT,CHZUSDT,MINAUSDT,IMXUSDT,FXSUSDT,ZECUSDT,CAKEUSDT,MKRUSDT
430,2023-05-06,28848.20,1896.33,321.9,0.4582,0.9993,0.3787,0.07672,0.9753,21.91,...,7.39,0.980,0.000095,0.1251,0.596,0.859,7.460,35.90,2.200,690.0
431,2023-05-07,28430.10,1870.40,321.0,0.4490,0.9994,0.3760,0.07652,0.9636,21.66,...,7.28,0.964,0.000095,0.1251,0.584,0.857,7.418,35.30,2.036,694.0
432,2023-05-08,27668.79,1847.56,313.8,0.4283,0.9995,0.3653,0.07239,0.9005,20.62,...,6.81,0.911,0.000084,0.1132,0.543,0.787,6.815,33.20,1.847,668.0
433,2023-05-09,27628.27,1846.52,312.1,0.4289,0.9999,0.3633,0.07310,0.8763,20.65,...,6.93,0.906,0.000089,0.1127,0.551,0.765,6.744,33.30,1.886,671.0
434,2023-05-10,27598.75,1840.84,314.3,0.4303,0.9997,0.3693,0.07312,0.8764,20.91,...,6.96,0.916,0.000093,0.1108,0.579,0.785,6.775,33.40,1.920,660.0
435,2023-05-11,26968.62,1794.30,307.5,0.4207,0.9998,0.3598,0.07193,0.8409,20.17,...,6.72,0.892,0.000086,0.1066,0.547,0.733,5.920,32.60,1.856,623.0
436,2023-05-12,26795.01,1807.45,308.5,0.4305,0.9998,0.3705,0.07235,0.8648,20.90,...,6.77,0.895,0.000091,0.1075,0.566,0.757,6.548,33.00,1.814,637.0
437,2023-05-13,26775.28,1795.11,310.7,0.4240,1.0000,0.3646,0.07167,0.8487,20.84,...,6.77,0.885,0.000089,0.1058,0.558,0.745,6.520,32.70,1.848,632.0
438,2023-05-14,26917.62,1799.21,312.1,0.4253,0.9998,0.3711,0.07196,0.8593,20.93,...,6.81,0.887,0.000090,0.1069,0.561,0.750,6.730,33.00,1.856,626.0
439,2023-05-15,27162.14,1816.20,313.4,0.4273,1.0000,0.3669,0.07191,0.8592,20.99,...,6.81,0.891,0.000090,0.1075,0.563,0.758,7.047,32.90,1.864,637.0


## Days from ATH / ATL (at_df)

### ATH

In [18]:
# Set Open_Time as Index
# All Time Df (at_df)
at_df = hist_prices.copy()
at_df.set_index('Open_Time', inplace=True)
at_df.head()

,BTCUSDT,ETHUSDT,BNBUSDT,XRPUSDT,BUSDUSDT,ADAUSDT,DOGEUSDT,MATICUSDT,SOLUSDT,DOTUSDT,...,AXSUSDT,XTZUSDT,LUNCUSDT,CHZUSDT,MINAUSDT,IMXUSDT,FXSUSDT,ZECUSDT,CAKEUSDT,MKRUSDT
Open_Time,,,,,,,,,,,,,,,,,,,,,
2022-03-02,43892.98,2947.30,409.3,0.7687,0.9996,0.939,0.1330,1.637,100.68,18.61,...,55.53,3.492,NaN,0.1970,2.139,1.465,23.448,125.8,6.83,1995.0
2022-03-03,42454.00,2833.99,402.6,0.7515,1.0001,0.903,0.1297,1.586,95.62,18.02,...,53.31,3.393,NaN,0.1971,2.036,1.459,24.420,129.1,6.58,2039.0
2022-03-04,39148.66,2622.10,374.2,0.7127,0.9998,0.844,0.1226,1.478,88.32,16.79,...,49.79,3.082,NaN,0.1953,2.008,1.323,22.303,117.0,5.99,1796.0
2022-03-05,39397.96,2665.15,384.7,0.7540,0.9999,0.864,0.1250,1.508,89.63,17.33,...,51.89,3.146,NaN,0.2328,2.021,1.282,22.280,116.4,6.20,1839.0
2022-03-06,38420.81,2551.29,375.0,0.7251,0.9998,0.823,0.1207,1.435,84.57,16.99,...,48.89,2.973,NaN,0.2075,1.912,1.148,20.587,107.4,6.08,1719.0


In [19]:
# #old code that was using limit 50
# i = 1
# all_time_high = pd.DataFrame()
# symbols = ['BTCUSDT', 'ETHUSDT','BNBUSDT','XRPUSDT','BUSDUSDT','ADAUSDT','DOGEUSDT','MATICUSDT','SOLUSDT','DOTUSDT','SHIBUSDT','LTCUSDT','TRXUSDT','AVAXUSDT','UNIUSDT','ATOMUSDT','LINKUSDT','ETCUSDT','XMRUSDT','BCHUSDT','HBARUSDT','XLMUSDT','APTUSDT','FILUSDT','LDOUSDT','NEARUSDT','APEUSDT','ALGOUSDT','VETUSDT','QNTUSDT','ICPUSDT','GRTUSDT','MANAUSDT','FTMUSDT','EOSUSDT','AAVEUSDT','SANDUSDT','EGLDUSDT','FLOWUSDT','THETAUSDT','AXSUSDT','XTZUSDT','LUNCUSDT','CHZUSDT','MINAUSDT','IMXUSDT','FXSUSDT','ZECUSDT','CAKEUSDT','MKRUSDT']

# for symbol in symbols:
#     ath=[]
#     while i in range(len(at_df[symbol])+1):
#         if at_df[symbol].iloc[:i].index[i-1]  == at_df[symbol].iloc[:i].idxmax():
#             ath.append("ATH")
#         else:
#             ath.append((at_df[symbol].iloc[:i].index[i-1]  - at_df[symbol].iloc[:i].idxmax()).days)         
#         i +=1
#     if len(ath) == 0:
#         all_time_high[symbol] = [np.nan] #* 484  # assign 49 NaN values if ath is empty
#     else:
#         all_time_high[symbol] = ath
#     i = 1  # reset i for the next symbol
# all_time_high.set_index(at_df.index, inplace=True) # Get the index from the at_df dataframe. Set Dates as Index
# all_time_high = all_time_high.add_suffix("-ATH") #Add suffix of "ATH" on all the columns
# all_time_high.head()

In [20]:
i = 1
all_time_high = pd.DataFrame()
symbols = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'XRPUSDT', 'BUSDUSDT', 'ADAUSDT', 'DOGEUSDT', 'MATICUSDT', 'SOLUSDT', 'DOTUSDT', 'SHIBUSDT', 'LTCUSDT', 'TRXUSDT', 'AVAXUSDT', 'UNIUSDT', 'ATOMUSDT', 'LINKUSDT', 'ETCUSDT', 'XMRUSDT', 'BCHUSDT', 'HBARUSDT', 'XLMUSDT', 'APTUSDT', 'FILUSDT', 'LDOUSDT', 'NEARUSDT', 'APEUSDT', 'ALGOUSDT', 'VETUSDT', 'QNTUSDT', 'ICPUSDT', 'GRTUSDT', 'MANAUSDT', 'FTMUSDT', 'EOSUSDT', 'AAVEUSDT', 'SANDUSDT', 'EGLDUSDT', 'FLOWUSDT', 'THETAUSDT', 'AXSUSDT', 'XTZUSDT', 'LUNCUSDT', 'CHZUSDT', 'MINAUSDT', 'IMXUSDT', 'FXSUSDT', 'ZECUSDT', 'CAKEUSDT', 'MKRUSDT']

for symbol in symbols:
    ath = []
    while i <= len(at_df[symbol]):
        if at_df[symbol].iloc[:i].index[i - 1] == at_df[symbol].iloc[:i].idxmax():
            ath.append("ATH")
        else:
            idxmax_value = at_df[symbol].iloc[:i].idxmax()
            if pd.isnull(idxmax_value):
                ath.append(np.nan)
            else:
                ath.append((at_df[symbol].iloc[:i].index[i - 1] - idxmax_value).days)
        i += 1
    if len(ath) == 0:
        all_time_high[symbol] = [np.nan]
    else:
        all_time_high[symbol] = ath
    i = 1

all_time_high.set_index(at_df.index, inplace=True)
all_time_high = all_time_high.add_suffix("-ATH")
all_time_high.head()

,BTCUSDT-ATH,ETHUSDT-ATH,BNBUSDT-ATH,XRPUSDT-ATH,BUSDUSDT-ATH,ADAUSDT-ATH,DOGEUSDT-ATH,MATICUSDT-ATH,SOLUSDT-ATH,DOTUSDT-ATH,...,AXSUSDT-ATH,XTZUSDT-ATH,LUNCUSDT-ATH,CHZUSDT-ATH,MINAUSDT-ATH,IMXUSDT-ATH,FXSUSDT-ATH,ZECUSDT-ATH,CAKEUSDT-ATH,MKRUSDT-ATH
Open_Time,,,,,,,,,,,,,,,,,,,,,
2022-03-02,ATH,ATH,ATH,ATH,ATH,ATH,ATH,ATH,ATH,ATH,...,ATH,ATH,NaN,ATH,ATH,ATH,ATH,ATH,ATH,ATH
2022-03-03,1,1,1,1,ATH,1,1,1,1,1,...,1,1,NaN,ATH,1,1,ATH,ATH,1,ATH
2022-03-04,2,2,2,2,1,2,2,2,2,2,...,2,2,NaN,1,2,2,1,1,2,1
2022-03-05,3,3,3,3,2,3,3,3,3,3,...,3,3,NaN,ATH,3,3,2,2,3,2
2022-03-06,4,4,4,4,3,4,4,4,4,4,...,4,4,NaN,1,4,4,3,3,4,3


### ATL

In [21]:
i = 1
all_time_low = pd.DataFrame()
symbols = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'XRPUSDT', 'BUSDUSDT', 'ADAUSDT', 'DOGEUSDT', 'MATICUSDT', 'SOLUSDT', 'DOTUSDT', 'SHIBUSDT', 'LTCUSDT', 'TRXUSDT', 'AVAXUSDT', 'UNIUSDT', 'ATOMUSDT', 'LINKUSDT', 'ETCUSDT', 'XMRUSDT', 'BCHUSDT', 'HBARUSDT', 'XLMUSDT', 'APTUSDT', 'FILUSDT', 'LDOUSDT', 'NEARUSDT', 'APEUSDT', 'ALGOUSDT', 'VETUSDT', 'QNTUSDT', 'ICPUSDT', 'GRTUSDT', 'MANAUSDT', 'FTMUSDT', 'EOSUSDT', 'AAVEUSDT', 'SANDUSDT', 'EGLDUSDT', 'FLOWUSDT', 'THETAUSDT', 'AXSUSDT', 'XTZUSDT', 'LUNCUSDT', 'CHZUSDT', 'MINAUSDT', 'IMXUSDT', 'FXSUSDT', 'ZECUSDT', 'CAKEUSDT', 'MKRUSDT']

for symbol in symbols:
    atl = []
    while i <= len(at_df[symbol]):
        if at_df[symbol].iloc[:i].index[i - 1] == at_df[symbol].iloc[:i].idxmin():
            atl.append("ATL")
        else:
            idxmin = at_df[symbol].iloc[:i].idxmin()
            if pd.isnull(idxmin):
                atl.append(np.nan)
            else:
                atl.append((at_df[symbol].iloc[:i].index[i - 1] - idxmin).days)
        i += 1
    if len(atl) == 0:
        all_time_low[symbol] = np.nan
    else:
        all_time_low[symbol] = atl
    i = 1

all_time_low.set_index(at_df.index, inplace=True)
all_time_low = all_time_low.add_suffix("-ATL")
all_time_low.head(484)

,BTCUSDT-ATL,ETHUSDT-ATL,BNBUSDT-ATL,XRPUSDT-ATL,BUSDUSDT-ATL,ADAUSDT-ATL,DOGEUSDT-ATL,MATICUSDT-ATL,SOLUSDT-ATL,DOTUSDT-ATL,...,AXSUSDT-ATL,XTZUSDT-ATL,LUNCUSDT-ATL,CHZUSDT-ATL,MINAUSDT-ATL,IMXUSDT-ATL,FXSUSDT-ATL,ZECUSDT-ATL,CAKEUSDT-ATL,MKRUSDT-ATL
Open_Time,,,,,,,,,,,,,,,,,,,,,
2022-03-02,ATL,ATL,ATL,ATL,ATL,ATL,ATL,ATL,ATL,ATL,...,ATL,ATL,NaN,ATL,ATL,ATL,ATL,ATL,ATL,ATL
2022-03-03,ATL,ATL,ATL,ATL,1,ATL,ATL,ATL,ATL,ATL,...,ATL,ATL,NaN,1,ATL,ATL,1,1,ATL,1
2022-03-04,ATL,ATL,ATL,ATL,2,ATL,ATL,ATL,ATL,ATL,...,ATL,ATL,NaN,ATL,ATL,ATL,ATL,ATL,ATL,ATL
2022-03-05,1,1,1,1,3,1,1,1,1,1,...,1,1,NaN,1,1,ATL,ATL,ATL,1,1
2022-03-06,ATL,ATL,2,2,4,ATL,ATL,ATL,ATL,2,...,ATL,ATL,NaN,2,ATL,ATL,ATL,ATL,2,ATL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-20,211,367,367,367,101,173,367,367,173,174,...,6,171,43,6,6,172,367,10,8,168
2023-06-21,212,368,368,368,102,174,368,368,174,175,...,7,172,44,7,7,173,368,11,9,169
2023-06-22,213,369,369,369,103,175,369,369,175,176,...,8,173,45,8,8,174,369,12,10,170


In [22]:
#Old code that was using limit 50
# i = 1
# all_time_low = pd.DataFrame()
# symbols = ['BTCUSDT', 'ETHUSDT','BNBUSDT','XRPUSDT','BUSDUSDT','ADAUSDT','DOGEUSDT','MATICUSDT','SOLUSDT','DOTUSDT','SHIBUSDT','LTCUSDT','TRXUSDT','AVAXUSDT','UNIUSDT','ATOMUSDT','LINKUSDT','ETCUSDT','XMRUSDT','BCHUSDT','HBARUSDT','XLMUSDT','APTUSDT','FILUSDT','LDOUSDT','NEARUSDT','APEUSDT','ALGOUSDT','VETUSDT','QNTUSDT','ICPUSDT','GRTUSDT','MANAUSDT','FTMUSDT','EOSUSDT','AAVEUSDT','SANDUSDT','EGLDUSDT','FLOWUSDT','THETAUSDT','AXSUSDT','XTZUSDT','LUNCUSDT','CHZUSDT','MINAUSDT','IMXUSDT','FXSUSDT','ZECUSDT','CAKEUSDT','MKRUSDT']

# for symbol in symbols:
#     atl=[]
#     while i in range(len(at_df[symbol])+1):
#         if at_df[symbol].iloc[:i].index[i-1]  == at_df[symbol].iloc[:i].idxmin():
#             atl.append("ATL")
#         else:
#             atl.append((at_df[symbol].iloc[:i].index[i-1]  - at_df[symbol].iloc[:i].idxmin()).days)         
#         i +=1
#     if len(atl) == 0:
#         all_time_low[symbol] = [np.nan] * 49  # assign 49 NaN values if ath is empty
#     else:
#         all_time_low[symbol] = atl
#     i = 1  # reset i for the next symbol
# all_time_low.set_index(at_df.index, inplace=True) # Get the index from the at_df dataframe. Set Dates as Index
# all_time_low = all_time_low.add_suffix("-ATL") #Add suffix of "ATH" on all the columns
# all_time_low.head(484)

## Base_Table_1

### Price Change Day / Week/ Month / Year

In [23]:
# def price_change(x):
#     base_table = pd.DataFrame()
#     base_table['Open_Time'] = x['Open_Time']
#     for i in x.columns[1:]: #For i in all columns 
#         base_table[i + '_Day_Change_%'] = pd.DataFrame(((x[i] / x[i].shift(1) - 1)*100).round(decimals=2))
#         base_table[i + '_Week_Change_%'] = pd.DataFrame(((x[i] / x[i].shift(7) - 1)*100).round(decimals=2))
#         base_table[i + '_Month_Change_%'] = pd.DataFrame(((x[i] / x[i].shift(30) - 1)*100).round(decimals=2))
#         base_table[i + '_Year_Change_%'] = pd.DataFrame(((x[i] / x[i].shift(365) - 1)*100).round(decimals=2))
#     return(base_table)

def price_change(x):
    base_table = pd.DataFrame()
    base_table['Open_Time'] = x['Open_Time']
    for i in x.columns[1:]: #For i in all columns 
        base_table[i + '_Day_Change_%'] = pd.DataFrame(((x[i] / x[i].shift(1) - 1)).round(decimals=2))
        base_table[i + '_Week_Change_%'] = pd.DataFrame(((x[i] / x[i].shift(7) - 1)).round(decimals=2))
        base_table[i + '_Month_Change_%'] = pd.DataFrame(((x[i] / x[i].shift(30) - 1)).round(decimals=2))
        base_table[i + '_Year_Change_%'] = pd.DataFrame(((x[i] / x[i].shift(365) - 1)).round(decimals=2))
    return(base_table)

In [24]:
price_change_df = price_change(hist_prices)
price_change_df.set_index('Open_Time', inplace=True)
price_change_df.head(10)

/var/folders/cg/vf981v4x5tz8b5szmpt19_xm0000gp/T/ipykernel_99876/4239824179.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  base_table[i + '_Year_Change_%'] = pd.DataFrame(((x[i] / x[i].shift(365) - 1)).round(decimals=2))
/var/folders/cg/vf981v4x5tz8b5szmpt19_xm0000gp/T/ipykernel_99876/4239824179.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  base_table[i + '_Day_Change_%'] = pd.DataFrame(((x[i] / x[i].shift(1) - 1)).round(decimals=2))
/var/folders/cg/vf981v4x5tz8b5szmpt19_xm0000gp/T/ipykernel_99876/4239824179.py:16: 

,BTCUSDT_Day_Change_%,BTCUSDT_Week_Change_%,BTCUSDT_Month_Change_%,BTCUSDT_Year_Change_%,ETHUSDT_Day_Change_%,ETHUSDT_Week_Change_%,ETHUSDT_Month_Change_%,ETHUSDT_Year_Change_%,BNBUSDT_Day_Change_%,BNBUSDT_Week_Change_%,...,ZECUSDT_Month_Change_%,ZECUSDT_Year_Change_%,CAKEUSDT_Day_Change_%,CAKEUSDT_Week_Change_%,CAKEUSDT_Month_Change_%,CAKEUSDT_Year_Change_%,MKRUSDT_Day_Change_%,MKRUSDT_Week_Change_%,MKRUSDT_Month_Change_%,MKRUSDT_Year_Change_%
Open_Time,,,,,,,,,,,,,,,,,,,,,
2022-03-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-03-03,-0.03,NaN,NaN,NaN,-0.04,NaN,NaN,NaN,-0.02,NaN,...,NaN,NaN,-0.04,NaN,NaN,NaN,0.02,NaN,NaN,NaN
2022-03-04,-0.08,NaN,NaN,NaN,-0.07,NaN,NaN,NaN,-0.07,NaN,...,NaN,NaN,-0.09,NaN,NaN,NaN,-0.12,NaN,NaN,NaN
2022-03-05,0.01,NaN,NaN,NaN,0.02,NaN,NaN,NaN,0.03,NaN,...,NaN,NaN,0.04,NaN,NaN,NaN,0.02,NaN,NaN,NaN
2022-03-06,-0.02,NaN,NaN,NaN,-0.04,NaN,NaN,NaN,-0.03,NaN,...,NaN,NaN,-0.02,NaN,NaN,NaN,-0.07,NaN,NaN,NaN
2022-03-07,-0.01,NaN,NaN,NaN,-0.02,NaN,NaN,NaN,0.01,NaN,...,NaN,NaN,-0.01,NaN,NaN,NaN,0.01,NaN,NaN,NaN
2022-03-08,0.02,NaN,NaN,NaN,0.03,NaN,NaN,NaN,0.00,NaN,...,NaN,NaN,-0.01,NaN,NaN,NaN,0.01,NaN,NaN,NaN
2022-03-09,0.08,-0.04,NaN,NaN,0.06,-0.07,NaN,NaN,0.03,-0.04,...,NaN,NaN,0.04,-0.09,NaN,NaN,0.09,-0.05,NaN,NaN
2022-03-10,-0.06,-0.07,NaN,NaN,-0.04,-0.08,NaN,NaN,-0.05,-0.08,...,NaN,NaN,-0.01,-0.07,NaN,NaN,-0.07,-0.13,NaN,NaN


### ATH to the base table

In [25]:
# Split the columns of df_A into groups of 4
coin_groups = [price_change_df.iloc[:, i:i+4] for i in range(0, price_change_df.shape[1], 4)]

# Create a new list to store the modified groups of df_A
coin_groups_ATH = []

# Loop through each group of columns in df_A
for i, group in enumerate(coin_groups):
    # Add the corresponding column from df_B
    if i < all_time_high.shape[1]:
        # Use concat to add the column from df_B to the end of the group
        new_group = pd.concat([group, all_time_high.iloc[:, i]], axis=1)
    else:
        # If there are no more columns in df_B, just add the group as is
        new_group = group
    # Append the modified group to the list
    coin_groups_ATH.append(new_group)

# Concatenate the modified groups back into a single dataframe
coin_groups_ATH = pd.concat(coin_groups_ATH, axis=1)
coin_groups_ATH.head(1)

,BTCUSDT_Day_Change_%,BTCUSDT_Week_Change_%,BTCUSDT_Month_Change_%,BTCUSDT_Year_Change_%,BTCUSDT-ATH,ETHUSDT_Day_Change_%,ETHUSDT_Week_Change_%,ETHUSDT_Month_Change_%,ETHUSDT_Year_Change_%,ETHUSDT-ATH,...,CAKEUSDT_Day_Change_%,CAKEUSDT_Week_Change_%,CAKEUSDT_Month_Change_%,CAKEUSDT_Year_Change_%,CAKEUSDT-ATH,MKRUSDT_Day_Change_%,MKRUSDT_Week_Change_%,MKRUSDT_Month_Change_%,MKRUSDT_Year_Change_%,MKRUSDT-ATH
Open_Time,,,,,,,,,,,,,,,,,,,,,
2022-03-02,NaN,NaN,NaN,NaN,ATH,NaN,NaN,NaN,NaN,ATH,...,NaN,NaN,NaN,NaN,ATH,NaN,NaN,NaN,NaN,ATH


### ATL to the base table

In [26]:
# Split the columns of df_A into groups of 5
coin_groups = [coin_groups_ATH.iloc[:, i:i+5] for i in range(0, coin_groups_ATH.shape[1], 5)]

# Create a new list to store the modified groups of df_A
coin_groups_ATH_ATL = []

# Loop through each group of columns in df_A
for i, group in enumerate(coin_groups):
    # Add the corresponding column from df_B
    if i < all_time_low.shape[1]:
        # Use concat to add the column from df_B to the end of the group
        new_group = pd.concat([group, all_time_low.iloc[:, i]], axis=1)
    else:
        # If there are no more columns in df_B, just add the group as is
        new_group = group
    # Append the modified group to the list
    coin_groups_ATH_ATL.append(new_group)

# Concatenate the modified groups back into a single dataframe
coin_groups_ATH_ATL = pd.concat(coin_groups_ATH_ATL, axis=1)
coin_groups_ATH_ATL.head(1)

,BTCUSDT_Day_Change_%,BTCUSDT_Week_Change_%,BTCUSDT_Month_Change_%,BTCUSDT_Year_Change_%,BTCUSDT-ATH,BTCUSDT-ATL,ETHUSDT_Day_Change_%,ETHUSDT_Week_Change_%,ETHUSDT_Month_Change_%,ETHUSDT_Year_Change_%,...,CAKEUSDT_Month_Change_%,CAKEUSDT_Year_Change_%,CAKEUSDT-ATH,CAKEUSDT-ATL,MKRUSDT_Day_Change_%,MKRUSDT_Week_Change_%,MKRUSDT_Month_Change_%,MKRUSDT_Year_Change_%,MKRUSDT-ATH,MKRUSDT-ATL
Open_Time,,,,,,,,,,,,,,,,,,,,,
2022-03-02,NaN,NaN,NaN,NaN,ATH,ATL,NaN,NaN,NaN,NaN,...,NaN,NaN,ATH,ATL,NaN,NaN,NaN,NaN,ATH,ATL


### Multi-Index

In [27]:
# Define the multi-index levels
multi_cols = ['BTCUSDT', 'ETHUSDT','BNBUSDT','XRPUSDT','BUSDUSDT','ADAUSDT','DOGEUSDT',
              'MATICUSDT','SOLUSDT','DOTUSDT','SHIBUSDT','LTCUSDT','TRXUSDT','AVAXUSDT',
              'UNIUSDT','ATOMUSDT','LINKUSDT','ETCUSDT','XMRUSDT','BCHUSDT','HBARUSDT',
              'XLMUSDT','APTUSDT','FILUSDT','LDOUSDT','NEARUSDT','APEUSDT','ALGOUSDT',
              'VETUSDT','QNTUSDT','ICPUSDT','GRTUSDT','MANAUSDT','FTMUSDT','EOSUSDT',
              'AAVEUSDT','SANDUSDT','EGLDUSDT','FLOWUSDT','THETAUSDT','AXSUSDT','XTZUSDT',
              'LUNCUSDT','CHZUSDT','MINAUSDT','IMXUSDT','FXSUSDT','ZECUSDT','CAKEUSDT','MKRUSDT']

# Create a list of tuples for the multi-index levels
multi_index = [(multi_cols[i//6], coin_groups_ATH_ATL.columns[i]) for i in range(len(coin_groups_ATH_ATL.columns))]

In [28]:
# Create the multi-index
coin_groups_ATH_ATL.columns = pd.MultiIndex.from_tuples(multi_index)
coin_groups_ATH_ATL.head(50)

BTCUSDT                                               \
           BTCUSDT_Day_Change_% BTCUSDT_Week_Change_% BTCUSDT_Month_Change_%   
Open_Time                                                                      
2022-03-02                  NaN                   NaN                    NaN   
2022-03-03                -0.03                   NaN                    NaN   
2022-03-04                -0.08                   NaN                    NaN   
2022-03-05                 0.01                   NaN                    NaN   
2022-03-06                -0.02                   NaN                    NaN   
2022-03-07                -0.01                   NaN                    NaN   
2022-03-08                 0.02                   NaN                    NaN   
2022-03-09                 0.08                 -0.04                    NaN   
2022-03-10                -0.06                 -0.07                    NaN   
2022-03-11                -0.02                 -0.01                    NaN   
2022-03-12                 0.00                 -0.01                    NaN   
2022-03-13                -0.03                 -0.02                    NaN   
2022-03-14                 0.05                  0.04                    NaN   
2022-03-15                -0.01                  0.01                    NaN   
2022-03-16                 0.05                 -0.02                    NaN   
2022-03-17                -0.00                  0.04                    NaN   
2022-03-18                 0.02                  0.08                    NaN   
2022-03-19                 0.01                  0.09                    NaN   
2022-03-20                -0.02                  0.09                    NaN   
2022-03-21                -0.01                  0.03                    NaN   
2022-03-22                 0.03                  0.08                    NaN   
2022-03-23                 0.01                  0.04                    NaN   
2022-03-24                 0.03                  0.08                    NaN   
2022-03-25                 0.01                  0.06                    NaN   
2022-03-26                 0.00                  0.05                    NaN   
2022-03-27                 0.05                  0.13                    NaN   
2022-03-28                 0.01                  0.15                    NaN   
2022-03-29                 0.01                  0.12                    NaN   
2022-03-30                -0.01                  0.10                    NaN   
2022-03-31                -0.03                  0.03                    NaN   
2022-04-01                 0.02                  0.04                   0.05   
2022-04-02                -0.01                  0.03                   0.08   
2022-04-03                 0.01                 -0.01                   0.19   
2022-04-04                 0.00                 -0.01                   0.18   
2022-04-05                -0.02                 -0.04                   0.18   
2022-04-06                -0.05                 -0.08                   0.14   
2022-04-07                 0.01                 -0.05                   0.12   
2022-04-08                -0.03                 -0.09                   0.01   
2022-04-09                 0.01                 -0.07                   0.08   
2022-04-10                -0.01                 -0.09                   0.09   
2022-04-11                -0.06                 -0.15                   0.02   
2022-04-12                 0.01                 -0.12                   0.06   
2022-04-13                 0.03                 -0.05                   0.04   
2022-04-14                -0.03                 -0.08                   0.02   
2022-04-15                 0.02                 -0.04                  -0.01   
2022-04-16                -0.00                 -0.06                  -0.01   
2022-04-17                -0.02                 -0.06                  -0.05   
2022-04-18              

# Final Table

In [29]:
base_table_1 = pd.DataFrame(coin_groups_ATH_ATL)
base_table_1.head()

BTCUSDT                                               \
           BTCUSDT_Day_Change_% BTCUSDT_Week_Change_% BTCUSDT_Month_Change_%   
Open_Time                                                                      
2022-03-02                  NaN                   NaN                    NaN   
2022-03-03                -0.03                   NaN                    NaN   
2022-03-04                -0.08                   NaN                    NaN   
2022-03-05                 0.01                   NaN                    NaN   
2022-03-06                -0.02                   NaN                    NaN   

                                                                      ETHUSDT  \
           BTCUSDT_Year_Change_% BTCUSDT-ATH BTCUSDT-ATL ETHUSDT_Day_Change_%   
Open_Time                                                                       
2022-03-02                   NaN         ATH         ATL                  NaN   
2022-03-03                   NaN           1         ATL                -0.04   
2022-03-04                   NaN           2         ATL                -0.07   
2022-03-05                   NaN           3           1                 0.02   
2022-03-06                   NaN           4         ATL                -0.04   

                                                                               \
           ETHUSDT_Week_Change_% ETHUSDT_Month_Change_% ETHUSDT_Year_Change_%   
Open_Time                                                                       
2022-03-02                   NaN                    NaN                   NaN   
2022-03-03                   NaN                    NaN                   NaN   
2022-03-04                   NaN                    NaN                   NaN   
2022-03-05                   NaN                    NaN                   NaN   
2022-03-06                   NaN                    NaN                   NaN   

            ...                CAKEUSDT                                      \
            ... CAKEUSDT_Month_Change_% CAKEUSDT_Year_Change_% CAKEUSDT-ATH   
Open_Time   ...                                                               
2022-03-02  ...                     NaN                    NaN          ATH   
2022-03-03  ...                     NaN                    NaN            1   
2022-03-04  ...                     NaN                    NaN            2   
2022-03-05  ...                     NaN                    NaN            3   
2022-03-06  ...                     NaN                    NaN            4   

                                     MKRUSDT                        \
           CAKEUSDT-ATL MKRUSDT_Day_Change_% MKRUSDT_Week_Change_%   
Open_Time                                                            
2022-03-02          ATL                  NaN                   NaN   
2022-03-03          ATL                 0.02                   NaN   
2022-03-04          ATL                -0.12                   NaN   
2022-03-05            1                 0.02                   NaN   
2022-03-06            2                -0.07                   NaN   

                                                                     \
           MKRUSDT_Month_Change_% MKRUSDT_Year_Change_% MKRUSDT-ATH   
Open_Time                                                             
2022-03-02                    NaN                   NaN         ATH   
2022-03-03                    NaN                   NaN         ATH   
2022-03-04                    NaN                   NaN           1   
2022-03-05                    NaN                   NaN           2   
2022-03-06                    NaN                   NaN           3   

                        
           MKRUSDT-ATL  
Open_Time               
2022-03-02         ATL  
2022-03-03           1  
2022-03-04         ATL  
2022-03-05           1  
2022-03-06         ATL  

[5 rows x 300 columns]

# Transform the Table to save it

## Remove Multi-Index Level

In [30]:
#Dropping the top level indexes and keep
base_table_1_new = base_table_1.droplevel(0, axis=1)
#base_table_1_new = base_table_1_new.reset_index()
base_table_1_new.head()

,BTCUSDT_Day_Change_%,BTCUSDT_Week_Change_%,BTCUSDT_Month_Change_%,BTCUSDT_Year_Change_%,BTCUSDT-ATH,BTCUSDT-ATL,ETHUSDT_Day_Change_%,ETHUSDT_Week_Change_%,ETHUSDT_Month_Change_%,ETHUSDT_Year_Change_%,...,CAKEUSDT_Month_Change_%,CAKEUSDT_Year_Change_%,CAKEUSDT-ATH,CAKEUSDT-ATL,MKRUSDT_Day_Change_%,MKRUSDT_Week_Change_%,MKRUSDT_Month_Change_%,MKRUSDT_Year_Change_%,MKRUSDT-ATH,MKRUSDT-ATL
Open_Time,,,,,,,,,,,,,,,,,,,,,
2022-03-02,NaN,NaN,NaN,NaN,ATH,ATL,NaN,NaN,NaN,NaN,...,NaN,NaN,ATH,ATL,NaN,NaN,NaN,NaN,ATH,ATL
2022-03-03,-0.03,NaN,NaN,NaN,1,ATL,-0.04,NaN,NaN,NaN,...,NaN,NaN,1,ATL,0.02,NaN,NaN,NaN,ATH,1
2022-03-04,-0.08,NaN,NaN,NaN,2,ATL,-0.07,NaN,NaN,NaN,...,NaN,NaN,2,ATL,-0.12,NaN,NaN,NaN,1,ATL
2022-03-05,0.01,NaN,NaN,NaN,3,1,0.02,NaN,NaN,NaN,...,NaN,NaN,3,1,0.02,NaN,NaN,NaN,2,1
2022-03-06,-0.02,NaN,NaN,NaN,4,ATL,-0.04,NaN,NaN,NaN,...,NaN,NaN,4,2,-0.07,NaN,NaN,NaN,3,ATL


## Append Dataframes

### (1) Create a column with the corresponding Symbol

In [31]:
# Your symbols list
symbols = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'XRPUSDT', 'BUSDUSDT', 'ADAUSDT', 'DOGEUSDT', 'MATICUSDT', 'SOLUSDT', 'DOTUSDT',
           'SHIBUSDT', 'LTCUSDT', 'TRXUSDT', 'AVAXUSDT', 'UNIUSDT', 'ATOMUSDT', 'LINKUSDT', 'ETCUSDT', 'XMRUSDT', 'BCHUSDT',
           'HBARUSDT', 'XLMUSDT', 'APTUSDT', 'FILUSDT', 'LDOUSDT', 'NEARUSDT', 'APEUSDT', 'ALGOUSDT', 'VETUSDT', 'QNTUSDT',
           'ICPUSDT', 'GRTUSDT', 'MANAUSDT', 'FTMUSDT', 'EOSUSDT', 'AAVEUSDT', 'SANDUSDT', 'EGLDUSDT', 'FLOWUSDT',
           'THETAUSDT', 'AXSUSDT', 'XTZUSDT', 'LUNCUSDT', 'CHZUSDT', 'MINAUSDT', 'IMXUSDT', 'FXSUSDT', 'ZECUSDT',
           'CAKEUSDT', 'MKRUSDT']

# Assuming your DataFrame is named base_table_1_new with the index as the 'date' column
base_table_1_new = base_table_1_new.reset_index()  # Resetting the index to convert 'date' column into a regular column

# Add columns for each symbol every 6th column
for i, symbol in enumerate(symbols):
    column_index = (i + 1) * 7  # Index of the column after every 6th column
    column_name = symbol
    base_table_1_new.insert(column_index, column_name, symbol)

# Set the 'date' column back as the index
base_table_1_new.set_index('Open_Time', inplace=True)

base_table_1_new.head()

,BTCUSDT_Day_Change_%,BTCUSDT_Week_Change_%,BTCUSDT_Month_Change_%,BTCUSDT_Year_Change_%,BTCUSDT-ATH,BTCUSDT-ATL,BTCUSDT,ETHUSDT_Day_Change_%,ETHUSDT_Week_Change_%,ETHUSDT_Month_Change_%,...,CAKEUSDT-ATH,CAKEUSDT-ATL,CAKEUSDT,MKRUSDT_Day_Change_%,MKRUSDT_Week_Change_%,MKRUSDT_Month_Change_%,MKRUSDT_Year_Change_%,MKRUSDT-ATH,MKRUSDT-ATL,MKRUSDT
Open_Time,,,,,,,,,,,,,,,,,,,,,
2022-03-02,NaN,NaN,NaN,NaN,ATH,ATL,BTCUSDT,NaN,NaN,NaN,...,ATH,ATL,CAKEUSDT,NaN,NaN,NaN,NaN,ATH,ATL,MKRUSDT
2022-03-03,-0.03,NaN,NaN,NaN,1,ATL,BTCUSDT,-0.04,NaN,NaN,...,1,ATL,CAKEUSDT,0.02,NaN,NaN,NaN,ATH,1,MKRUSDT
2022-03-04,-0.08,NaN,NaN,NaN,2,ATL,BTCUSDT,-0.07,NaN,NaN,...,2,ATL,CAKEUSDT,-0.12,NaN,NaN,NaN,1,ATL,MKRUSDT
2022-03-05,0.01,NaN,NaN,NaN,3,1,BTCUSDT,0.02,NaN,NaN,...,3,1,CAKEUSDT,0.02,NaN,NaN,NaN,2,1,MKRUSDT
2022-03-06,-0.02,NaN,NaN,NaN,4,ATL,BTCUSDT,-0.04,NaN,NaN,...,4,2,CAKEUSDT,-0.07,NaN,NaN,NaN,3,ATL,MKRUSDT


### Create a dataframe which will have the performance of all different cryptos underneath eachother

In [32]:
# Reseting the Index before the loop
base_table_1_new.reset_index(inplace=True)

# Get the column names excluding the 'Open_Time' column
column_names = base_table_1_new.columns[1:]

# Initialize an empty DataFrame to store the concatenated subsets
result_df = pd.DataFrame()

for i in range(0, len(column_names), 7):
    subset_columns = ['Open_Time'] + [col.strip() for col in column_names[i:i+7]]
    subset_df = base_table_1_new[subset_columns]
    subset_df.columns = ['Open_Time', 'Day_Change', 'Week_Change', 'Month_Change','Year_Change','ATH','ATL','symbol']

result_df.head()

/var/folders/cg/vf981v4x5tz8b5szmpt19_xm0000gp/T/ipykernel_99876/3917417403.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  base_table_1_new.reset_index(inplace=True)


""


In [33]:
result_df.to_csv("result_df.csv")

# Save Table

In [34]:
spark_base_table_1_new = spark.createDataFrame(result_df) 
spark_base_table_1_new.createOrReplaceTempView("base_table_1_new")

NameError: name 'spark' is not defined

In [ ]:
%sql
select *
from base_table_1_new

In [ ]:
%sql
USE CATALOG cryptoproject;
USE SCHEMA bexapo;
CREATE table IF NOT EXISTS base_table_1 
select * from base_table_1_new
;
select * from cryptoproject.bexapo.base_table_1

In [ ]:
# -- If the data we want to insert to the table is not already existing, then insert
# insert into bexapo.uplift_demand
# select * 
# from demand_per_month dpm
# where not exists (
# select * 
# from bexapo.uplift_demand ud
# where dpm.event_territorry = ud.event_territorry and dpm.month_of_event = ud.month_of_event
# )
# ;
# select * from bexapo.uplift_demand